In [4]:
import os

os.environ["SEED"] = "0"

import json
from pathlib import Path

import numpy as np
import pandas as pd

from src.constant import DATA_DIR, DATABASE_DIR
from src.database import DB
from src.instance.BBOB_Instance import BBOB_Instance, set_08_cut_off_time
from src.instance.InstanceList import InstanceList

pd.options.display.max_columns = None

In [15]:
instance = BBOB_Instance(function_index=1, dimension=2, instance_index=1)

In [ ]:
n = 25
seed = 0

rng = np.random.default_rng(seed=seed)
function_index_list = list(range(1, 25))
dimension_list = [2, 3, 5, 10, 20]
instance_index_list = [3, 4, 5]
n_per_dimension = n // len(dimension_list)
instance_list = InstanceList()

for dimension in dimension_list:
    sampled_pairs = rng.choice(
        len(function_index_list) * len(instance_index_list),
        size=n_per_dimension,
        replace=False
    )

    for idx in sampled_pairs:
        function_idx = function_index_list[idx // len(instance_index_list)]
        instance_idx = instance_index_list[idx % len(instance_index_list)]
        
        bbob_instance = BBOB_Instance(
            function_index=function_idx,
            dimension=dimension,
            instance_index=instance_idx
        )
        instance_list.append(bbob_instance)

instance_list

InstanceList(size=25)[BBOB_Instance(problem_id=bbob_f015_i04_d02), BBOB_Instance(problem_id=bbob_f012_i05_d02), BBOB_Instance(problem_id=bbob_f007_i04_d02), BBOB_Instance(problem_id=bbob_f008_i04_d02), BBOB_Instance(problem_id=bbob_f020_i03_d02), BBOB_Instance(problem_id=bbob_f019_i04_d03), BBOB_Instance(problem_id=bbob_f012_i05_d03), BBOB_Instance(problem_id=bbob_f015_i05_d03), BBOB_Instance(problem_id=bbob_f022_i03_d03), BBOB_Instance(problem_id=bbob_f015_i04_d03), BBOB_Instance(problem_id=bbob_f017_i03_d05), BBOB_Instance(problem_id=bbob_f020_i03_d05), BBOB_Instance(problem_id=bbob_f007_i04_d05), BBOB_Instance(problem_id=bbob_f022_i04_d05), BBOB_Instance(problem_id=bbob_f013_i05_d05), BBOB_Instance(problem_id=bbob_f018_i03_d10), BBOB_Instance(problem_id=bbob_f018_i04_d10), BBOB_Instance(problem_id=bbob_f005_i03_d10), BBOB_Instance(problem_id=bbob_f021_i03_d10), BBOB_Instance(problem_id=bbob_f001_i05_d10), BBOB_Instance(problem_id=bbob_f007_i05_d20), BBOB_Instance(problem_id=bbob_f01

In [ ]:
# features = {}

# for features_file in Path("features").glob("*.json"):
#     with open(features_file, "r") as f:
#         features = {
#             **features,
#             **json.load(f)
#         }

# len(features)

# with open(DATA_DIR / "BBOB" / "features.json", "w") as f:
#     json.dump(features, f, indent=4)

In [ ]:
# from src.instance.BBOB_Instance import BBOB_Instance
# import json

# features = {}

# for instance_index in [1, 2, 3, 4, 5]:
#     instance = BBOB_Instance(
#         function_index=7,
#         dimension=2,
#         instance_index=instance_index,
#     )
#     result, time = instance.calculate_features()
#     features[instance.id()] = {"result": result, "time": time}

# with open("features/features_7_2.json", "w") as f:
#     json.dump(features, f, indent=4)